In [ ]:
!pip install oracledb

import oracledb
from datetime import datetime

# Defining Connection and Deisconnecting Functions
##########################################################################################################
def oracleConnect (username, password, dsn_link = "vu2025.cypibltd7eim.us-east-2.rds.amazonaws.com/ORCL"):
    conn = oracledb.connect(user = username, password = password, dsn = dsn_link)
    print("Connected to Oracle Database.  Version: ", conn.version)
    return conn

def oracleDisconnect (conn):
    conn.close()
    print("Disconnected from Oracle Database.")
##########################################################################################################


# Defining Functions that perform the 
    # a.Read some data from the database and display it to the user
    #b.	Change some data in the database (insert, update, or delete)
    #c.	Call one of your stored procedures

##########################################################################################################

# Function to print all employees with their details
def printAllEmployees (conn):
    cursor = conn.cursor()
    sql = """
        SELECT 
            e.employeeId,
            e.firstName,
            e.lastName,
            e.email,
            e.phone,
            d.name AS divisionName,
            r.title AS roleName,
            m.firstName || ' ' || m.lastName AS managerName
        FROM 
            Employee e
        JOIN 
            Division d ON e.division = d.divisionId
        JOIN 
            Role r ON e.role = r.roleId
        LEFT JOIN 
            Employee m ON e.manager = m.employeeId
    """
    for result in cursor.execute(sql):
        print(result)
    cursor.close()

# Function to print all projects with their details
def printAllProjects (conn):
    cursor = conn.cursor()
    sql = """
        SELECT 
            p.projectId,
            p.name AS projectName,
            p.startdate,
            p.enddate,
            p.street,
            p.district,
            p.province,
            p.zip
        FROM 
            Project p
    """
    for result in cursor.execute(sql):
        print(result)
    cursor.close()

##########################################################################################################
# Function to print all projects with their total budget
def printAllProjectsTotalBudget (conn):
    cursor = conn.cursor()
    sql = """
        SELECT 
            p.projectId,
            p.name AS projectName,
            SUM(b.amountAllocated) AS totalAllocated
        FROM 
            Project p
        JOIN 
            Budget b ON p.projectId = b.project
        GROUP BY 
            p.projectId, p.name
    """
    for result in cursor.execute(sql):
        print(result)
    cursor.close()
##########################################################################################################
# Function to insert a new project into the database
def insertNewProject(conn, projectId, name, startdate, enddate, street, district, province, zip):
    cursor = conn.cursor()
    sql = """
        INSERT INTO Project (projectId, name, startdate, enddate, street, district, province, zip)
        VALUES (:projectId, :name, :startdate, :enddate, :street, :district, :province, :zip)
    """
    cursor.execute(sql, projectId=projectId, name=name, startdate=startdate,
                   enddate=enddate, street=street, district=district,
                   province=province, zip=zip)
    conn.commit()
    print("Inserted new project with ID:", projectId)
    cursor.close()

# Function to update project detail in the database
def updateProjectDetail(conn, projectId, name, startdate, enddate, street, district, province, zip):
    cursor = conn.cursor()
    sql = """
        UPDATE Project
        SET name = :name,
            startdate = :startdate,
            enddate = :enddate,
            street = :street,
            district = :district,
            province = :province,
            zip = :zip
        WHERE projectId = :projectId
    """
    cursor.execute(sql, projectId=projectId, name=name, startdate=startdate,
                   enddate=enddate, street=street, district=district,
                   province=province, zip=zip)
    conn.commit()
    print("Updated project with ID:", projectId)
    cursor.close()


# Function to delete a project from the database

def deleteProject(conn, projectId):
    cursor = conn.cursor()
    sql = """
        DELETE FROM Project
        WHERE projectId = :projectId
    """
    cursor.execute(sql, projectId=projectId)
    conn.commit()
    print("Deleted project with ID:", projectId)
    cursor.close()

##########################################################################################################
# Function to call the stored procedure to add a new employee
def call_add_new_employee_procedure(conn, empId, firstName, lastName, email, phone, division, role, manager):
    cursor = conn.cursor()
    cursor.callproc("add_new_employee_with_login", [empId, firstName, lastName, email, phone, division, role, manager])
    conn.commit()
    cursor.close()

##########################################################################################################
conn = oracleConnect('srijal', '02609941')
##########################################################################################################

# a. Read some data from database and display it to user
print("---------------------------------------------------------------------------\n\n")
print("Demonstrating SELECT Query usage: Total Budget for each project:\n")
printAllProjectsTotalBudget(conn)
print("---------------------------------------------------------------------------\n\n")
##########################################################################################################

# Initially
print("---------------------------------------------------------------------------\n\n")
print("All Projects Before Insertion of New Project:\n")
printAllProjects(conn)
print("---------------------------------------------------------------------------\n\n")

# Insert
startdate = datetime.strptime('2023-10-01', '%Y-%m-%d').date()
enddate = datetime.strptime('2024-10-01', '%Y-%m-%d').date()
insertNewProject(conn, 'P010', 'New Project', startdate, enddate, '123 Main St', 'Downtown', 'NY', '10001')

# After Insert
print("---------------------------------------------------------------------------\n\n")
print("All Projects After Insertion of New Project:\n")
printAllProjects(conn)
print("---------------------------------------------------------------------------\n\n")

# Update
startdate = datetime.strptime('2023-11-01', '%Y-%m-%d').date()
enddate = datetime.strptime('2024-11-01', '%Y-%m-%d').date()
updateProjectDetail(conn, 'P010', 'Updated Project', startdate, enddate, '456 Elm St', 'Uptown', 'NY', '10002')

# After Update
print("---------------------------------------------------------------------------\n\n")
print("All Projects After Updating Project:\n")
printAllProjects(conn)
print("---------------------------------------------------------------------------\n\n")

# Delete
deleteProject(conn, 'P010')

# After Delete
print("---------------------------------------------------------------------------\n\n")
print("All Projects After Deleting Project:\n")
printAllProjects(conn)
print("---------------------------------------------------------------------------\n\n")

##########################################################################################################
# Calling stored procedure

# Initially
print("---------------------------------------------------------------------------\n\n")
print("All Employees Before Calling Stored Procedure:\n")
printAllEmployees(conn)
print("---------------------------------------------------------------------------\n\n")

# Call the stored procedure to add a new employee
call_add_new_employee_procedure(conn, 'E010', 'Johny', 'Tom', 'johny.tom@what.com', '3724563452', 'D001', 'R001', 'E002')

# After calling the stored procedure
print("---------------------------------------------------------------------------\n\n")
print("All Employees After Calling Stored Procedure:\n")
printAllEmployees(conn)
print("---------------------------------------------------------------------------\n\n")

##########################################################################################################
oracleDisconnect(conn)

13464.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Connected to Oracle Database.  Version:  19.26.0.0.0
---------------------------------------------------------------------------


Total Budget for each project:

('P006      ', 'Airport Terminal', 12000000)
('P005      ', 'Shopping Mall', 10000000)
('P002      ', 'Hospital Renovation', 3500000)
('P001      ', 'City Road Expansion', 5500000)
('P003      ', 'Bridge Construction', 7500000)
('P004      ', 'Residential Complex', 6000000)
---------------------------------------------------------------------------


---------------------------------------------------------------------------


All Projects Before Insertion of New Project:

('P001      ', 'City Road Expansion', datetime.datetime(2025, 1, 1, 0, 0), datetime.datetime(2025, 12, 31, 0, 0), 'Baker Street Revised', 'Westminster', 'London', 'W1A1A')
('P002      ', 'Hospital Renovation', datetime.datetime(2025, 3, 1, 0, 0), datetime.datetime(2026, 2, 28, 0, 0), 'MG Road', 'Andheri', 'Mumbai', '40058')
('P003      ', 'Bridge Constructi